# La Bibliografía Española de Cartografía

Este notebook utiliza la Bibliografía Española de Cartografía es una publicación cuyo objetivo principal es dar a conocer el material cartográfico publicado en España, que ingresa en la Biblioteca Nacional de España que incluye mapas, planos, cartas náuticas, atlas, etc., tanto en formato impreso como electrónico. 

En el año 2007 se inició la publicación en línea y desde el 2010 se incorporan a esta bibliografía los atlas, antes recogidos en la Bibliografía Española de Monografías. Se publica anualmente y es consultable en línea.

https://datos.gob.es/es/catalogo/e00123904-bibliografia-espanola-de-cartografia-2017

### Importando las librerías de código

In [1]:
# https://pypi.org/project/pymarc/
import pymarc, re, csv
import pandas as pd
from pymarc import parse_xml_to_array
from datapackage import Package

### Generando un fichero CSV como salida con el contenido procesado a partir de los ficheros originales

In [28]:
with open('registros_marc_bne.csv', 'w') as csv_sal:
    csv_salida = csv.writer(csv_sal, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    csv_salida.writerow(['titulo', 'autor', 'variante_titulo', 'extension', 'distribuidor', 'materias', 'nota', 'detalles'])

    registros = parse_xml_to_array(open('BNE-cartografia/BN_CARTOGRAFIA_2017-MARCXML.xml'))

    for registro in registros:

        titulo = autor = variante_titulo = extension = distribuidor = materias = nota = detalles =''

        # titulo
        if registro['245'] is not None:
          titulo = registro['245']['a']
          if registro['245']['b'] is not None:
            titulo = titulo + " " + registro['245']['b']

        # autor
        if registro['100'] is not None:
          autor = registro['100']['a']
        elif registro['110'] is not None:
          autor = registro['110']['a']
        elif registro['700'] is not None:
          autor = registro['700']['a']
        elif registro['710'] is not None:
          autor = registro['710']['a']

        # variante de titulo
        if registro['246'] is not None:
          variante_titulo = registro['246']['a']


        # Physical Description - extent
        for f in registro.get_fields('300'):
            extension = f.get_subfields('a')
            if len(extension):
                extension = extension[0]
                # TODO cleaning
            detalles = f.get_subfields('b')
            if len(detalles):
                detalles = detalles[0]

        # distribuidor
        if registro['260'] is not None:
          distribuidor = registro['260']['b']

        # nota
        if registro['501'] is not None:
          nota = registro['501']['a']

        # materia
        if registro['650'] is not None:
            materias = '' 
            for f in registro.get_fields('650'):
                materias += f.get_subfields('a')[0] + ' -- '
                materias += f.get_subfields('v')[0] + ' -- '

            materias = re.sub(' -- $', '', materias)

        csv_salida.writerow([titulo,autor,variante_titulo,extension,distribuidor,materias,nota,detalles])

## Leyendo el fichero CSV 

In [29]:
# Este comando añade el contenido del fichero a un Pandas DataFrame
df = pd.read_csv('registros_marc_bne.csv')

## Consultando el contenido

In [30]:
df

,titulo,autor,variante_titulo,extension,distribuidor,materias,nota,detalles
0,14 routen um Manacor,Manacor,Vierzehn routen um Manacor,1 mapa,"Ajuntament de Manacor, Delegació de Turisme",Excursionismo -- Mapas,Incluye fotografías,col.
1,14 routes in Manacor,Manacor,Fourteen routes in Manacor,1 mapa,"Ajuntament de Manacor, Delegació de Turisme",Excursionismo -- Mapas,Incluye fotografías,col.
2,14 rutas por Manacor,Manacor,Catorce rutas por Manacor,1 mapa,"Ajuntament de Manacor, Delegació de Turisme",Excursionismo -- Mapas,Incluye fotografías,col.
3,14 rutes per Manacor,Manacor,Quatorze rutes per Manacor,1 mapa,"Ajuntament de Manacor, Delegació de Turisme",Excursionismo -- Mapas,Incluye fotografías,col.
4,A Coruña,Turismo de Galicia,NaN,1 plano,Turismo de Galicia,NaN,Al verso: Información y localización en el pla...,col.
...,...,...,...,...,...,...,...,...
1298,[Southeastern Spain] low flying chart : Europe,España,NaN,1 h.,"Ministerio de Defensa, Secretaría General Técnica",NaN,Al verso: Clave por símbolos; frecuencias de a...,col.
1299,[Southwestern Spain] low flying chart : Europe,España,NaN,1 h.,"Ministerio de Defensa, Secretaría General Técnica",NaN,Al verso: Clave por símbolos; información aero...,col.
1300,[Southwestern Spain] low flying chart : Europe,España,NaN,1 h.,"Ministerio de Defensa, Secretaría General Técnica",NaN,Al verso: Clave por símbolos; frecuencias de a...,col.
1301,[Western Spain] low flying chart : Europe,España,NaN,1 h.,"Ministerio de Defensa, Secretaría General Técnica",NaN,Al verso: Clave por símbolos; información aero...,col.


### Consultando las columnas

In [22]:
df.columns

Index(['titulo', 'autor', 'variante_titulo', 'extension', 'distribuidor',
       'materias', 'nota', 'detalles'],
      dtype='object')

### ¿Cuántos registros existen?

In [23]:
len(df)

1303

## Explorando las materias
### Creamos una lista de materias y la ordenamos alfabéticamente

In [24]:
df['materias'][2]

'Excursionismo -- Mapas'

In [25]:
df['materias'].str.split(' -- ', expand=True).stack()

0     0    Excursionismo
      1            Mapas
1     0    Excursionismo
      1            Mapas
2     0    Excursionismo
               ...      
1284  1           Planos
1285  0        Comercios
      1           Planos
1295  0       Senderismo
      1            Mapas
Length: 1198, dtype: object

In [26]:
# Obtener valores únicos
materias = pd.unique(df['materias'].str.split(' -- ', expand=True).stack()).tolist()
for materia in sorted(materias, key=str.lower):
    print(materia)

Aceite de oliva
Actividades recreativas al aire libre
Albergues juveniles
Alojamientos turísticos
Arquitectura
Arquitectura modernista
Arquitectura románica
Arte románico
Aves
Bares
Bodegas
Buceo
Caminos
Carreras campo a través (Atletismo)
Carreras ciclistas
Carreras de automóviles
Carreteras
Centrales hidroeléctricas
Centros docentes
Ciclismo
Ciclismo todoterreno
Cicloturismo
Cine
Cines
Comarcas
Comercios
Costas
Deportes acuáticos
Desplazamientos en bicicleta
Enoturismo
Escuelas infantiles
Excursionismo
Ferrocarriles de alta velocidad
Ferrocarriles metropolitanos
Geología
Geomorfología
Geoparques
Hidrogeología
Hidrología
Historia
Hostelería
Jardines botánicos
Lengua vasca
Mantequilla
Mapas
Mapas 
Mitología
Museos
Nombres geográficos
Orientación (Deporte)
Paisaje
Parques naturales
Peregrinaciones cristianas
Pesca de agua dulce
Piragüismo
Planos
Quesos
Radiación natural
Recogida selectiva de residuos
Restaurantes
Rutas literarias
Sector servicios
Senderismo
Sidra
Suelos
Surf
Transportes

### También podemos calcular con qué frecuencia se usa una materia

In [27]:
# Partir las materias y obterner el número de ocurrencias
materia_contador = df['materias'].str.split(' -- ').apply(lambda x: pd.Series(x).value_counts()).sum().astype('int').sort_values(ascending=False).to_frame().reset_index(level=0)
# Añadimos las columnas
materia_contador.columns = ['materia', 'contador']
# Mostrar con barras horizontales
display(materia_contador.style.bar(subset=['contador'], color='#d65f5f').set_properties(subset=['contador'], **{'width': '300px'}))

,materia,contador
0,Mapas,508
1,Excursionismo,130
2,Carreteras,95
3,Planos,88
4,Senderismo,81
5,Comercios,79
6,Geología,22
7,Nombres geográficos,22
8,Cicloturismo,17
9,Suelos,12
